In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

# random
import random

# numpy
import numpy

# classifier
from sklearn.linear_model import LogisticRegression

import logging
import sys

In [2]:
log = logging.getLogger()
log.setLevel(logging.INFO)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
log.addHandler(ch)

class TaggedLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return(self.sentences)

    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return(shuffled)


In [3]:
log.info('source load')
sources = {'test-neg.txt':'TEST_NEG', 'test-pos.txt':'TEST_POS', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS', 'train-unsup.txt':'TRAIN_UNS'}

log.info('TaggedDocument')
sentences = TaggedLineSentence(sources)

log.info('D2V')
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

2017-07-18 09:52:52,429 - root - INFO - source load
2017-07-18 09:52:52,431 - root - INFO - TaggedDocument
2017-07-18 09:52:52,432 - root - INFO - D2V
2017-07-18 09:53:00,808 - gensim.models.doc2vec - INFO - collecting all words and their counts
2017-07-18 09:53:00,809 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-07-18 09:53:02,705 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 2338931 words (1233667/s), 48913 word types, 10000 tags
2017-07-18 09:53:03,804 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 4709174 words (2160221/s), 66801 word types, 20000 tags
2017-07-18 09:53:04,775 - gensim.models.doc2vec - INFO - PROGRESS: at example #30000, processed 7117318 words (2481777/s), 80306 word types, 30000 tags
2017-07-18 09:53:05,610 - gensim.models.doc2vec - INFO - PROGRESS: at example #40000, processed 9476091 words (2830972/s), 91324 word types, 40000 tags
2017-07-18 09:53:0

In [5]:
log.info('Epoch')
for epoch in range(10):
    log.info('EPOCH: {}'.format(epoch))
    model.train(sentences.sentences_perm(),total_examples=model.corpus_count,
    epochs=model.iter)

2017-07-18 09:54:44,704 - root - INFO - Epoch
2017-07-18 09:54:44,706 - root - INFO - EPOCH: 0
2017-07-18 09:54:44,902 - gensim.models.word2vec - INFO - training model with 7 workers on 139893 vocabulary and 100 features, using sg=0 hs=0 sample=0.0001 negative=5 window=10
2017-07-18 09:54:45,915 - gensim.models.word2vec - INFO - PROGRESS: at 0.49% examples, 302712 words/s, in_qsize 13, out_qsize 0
2017-07-18 09:54:46,921 - gensim.models.word2vec - INFO - PROGRESS: at 0.92% examples, 275090 words/s, in_qsize 13, out_qsize 0
2017-07-18 09:54:47,924 - gensim.models.word2vec - INFO - PROGRESS: at 1.49% examples, 293669 words/s, in_qsize 13, out_qsize 0
2017-07-18 09:54:48,928 - gensim.models.word2vec - INFO - PROGRESS: at 1.95% examples, 288448 words/s, in_qsize 13, out_qsize 0
2017-07-18 09:54:49,931 - gensim.models.word2vec - INFO - PROGRESS: at 2.42% examples, 285493 words/s, in_qsize 13, out_qsize 0
2017-07-18 09:54:50,936 - gensim.models.word2vec - INFO - PROGRESS: at 2.88% examples, 

In [6]:
log.info('Model Save')
model.save('./imdb.d2v')
model = Doc2Vec.load('./imdb.d2v')

2017-07-18 10:26:02,507 - root - INFO - Model Save
2017-07-18 10:26:02,508 - gensim.utils - INFO - saving Doc2Vec object under ./imdb.d2v, separately None
2017-07-18 10:26:02,510 - gensim.utils - INFO - not storing attribute syn0norm
2017-07-18 10:26:02,511 - gensim.utils - INFO - storing np array 'syn0' to ./imdb.d2v.wv.syn0.npy
2017-07-18 10:26:02,974 - gensim.utils - INFO - not storing attribute cum_table
2017-07-18 10:26:02,975 - gensim.utils - INFO - storing np array 'syn1neg' to ./imdb.d2v.syn1neg.npy
2017-07-18 10:26:04,761 - gensim.utils - INFO - saved ./imdb.d2v
2017-07-18 10:26:04,762 - gensim.utils - INFO - loading Doc2Vec object from ./imdb.d2v
2017-07-18 10:26:06,316 - gensim.utils - INFO - loading wv recursively from ./imdb.d2v.wv.* with mmap=None
2017-07-18 10:26:06,316 - gensim.utils - INFO - loading syn0 from ./imdb.d2v.wv.syn0.npy with mmap=None
2017-07-18 10:26:06,352 - gensim.utils - INFO - setting ignored attribute syn0norm to None
2017-07-18 10:26:06,354 - gensim.

In [7]:
log.info('Sentiment')
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

log.info(train_labels)

2017-07-18 10:26:10,949 - root - INFO - Sentiment
2017-07-18 10:26:11,083 - root - INFO - [ 1.  1.  1. ...,  0.  0.  0.]


In [8]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[12500 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

log.info('Fitting')
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

2017-07-18 10:26:25,175 - root - INFO - Fitting


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

log.info(classifier.score(test_arrays, test_labels))

2017-07-18 10:26:31,770 - root - INFO - 0.86472
